In [13]:
# list all directories in 
path="../../discoveries"

import os
import json

from PIL import Image
import cv2
#from pydub import AudioSegment

discoveries = []

#list all directories in path
for root, dirs, files in os.walk(path):
    for name in dirs:
        discovery={}

        #load discovery.json file
        with open(os.path.join(root, name, 'discovery.json')) as f:
            discovery_details = json.load(f)
        
        discovery_embedding=discovery_details['output']
        
        #list all files ending with .json
        for file in os.listdir(os.path.join(root, name)):
            if file == "discovery.json":
                discovery['name']=name
                discovery['embedding']=discovery_embedding
                discoveries.append(discovery)

                break

print(len(discoveries))

14657


In [14]:
import numpy as np
X = np.array([discovery['embedding'] for discovery in discoveries])
# import umap
# reducer = umap.UMAP()
# embedding = reducer.fit_transform(X)

In [15]:
# get the most isolated discovery
from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=2).fit(X)
distances, indices = nbrs.kneighbors(X)

max_distance = 0
max_distance_index = 0
for i in range(len(distances)):
    if distances[i][1] > max_distance:
        max_distance = distances[i][1]
        max_distance_index = i

print(max_distance)
print(max_distance_index)
print(discoveries[max_distance_index])

0.007820591708820207
7100
{'name': '2024-10-19T23:36_exp_0_idx_15_seed_42', 'embedding': [0.9892776835298216, 0.0013876937997388333, 0.0008504638541287589, 0.0005720048899856959, 0.0004365806706304045, 0.0003530314825352423, 0.0002601893775493787, 0.0002478356668804894, 0.00023057693978769726, 0.000260527308057621, 0.0002388532276068022, 0.00023526539103880617, 0.00019011200613902943, 0.00019604724202091565, 0.00019011200613902943, 0.00023526539103880614, 0.0002388532276068022, 0.00026052730805762106, 0.00023057693978769726, 0.0002478356668804893, 0.0002601893775493787, 0.0003530314825352424, 0.0004365806706304045, 0.0005720048899856956, 0.0008504638541287589, 0.001387693799738833]}


In [111]:
discoveries=discoveries

In [113]:
#PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca.fit(X)
embedding = pca.transform(X)


In [114]:
#TSNE
# from sklearn.manifold import TSNE
# tsne = TSNE(n_components=2, random_state=0,perplexity= len(discoveries)-1)
# embedding = tsne.fit_transform(X)

In [115]:
for i, discovery in enumerate(discoveries):
    del discovery['embedding']
    discovery['x'] = embedding[i][0].item()
    discovery['y'] = embedding[i][1].item()

In [116]:
#normalize x and y between -1/2 and 1/2
min_x = min(discovery['x'] for discovery in discoveries)
max_x = max(discovery['x'] for discovery in discoveries)
min_y = min(discovery['y'] for discovery in discoveries)
max_y = max(discovery['y'] for discovery in discoveries)

for discovery in discoveries:
    discovery['x'] = (discovery['x'] - min_x) / (max_x - min_x) - 0.5
    discovery['y'] = (discovery['y'] - min_y) / (max_y - min_y) - 0.5

#normalize x and y between -1 and 1
# for discovery in discoveries:
#     discovery['x'] = (discovery['x'] - min_x) / (max_x - min_x) * 2 - 1
#     discovery['y'] = (discovery['y'] - min_y) / (max_y - min_y) * 2 - 1

